In [1]:
%pylab inline
num_samples = 44100
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

Populating the interactive namespace from numpy and matplotlib


In [22]:
!cat 01_simple.stride

use DesktopAudio version 1.0

switch AlwaysOn {
	default: on
	domain: AudioDomain
}


reaction ConstantSwitch {
	ports: [
		mainOutputPort OutputPort {
			block: ReactionOutput
		}
	]
	streams: [
		1.8 >> ReactionOutput;
	]
}

AlwaysOn >> ConstantSwitch() >> AudioOut[0];
	


In [23]:

out_text = ''

value = 0.0
    

for val1, val2 in zip(input1, input2):
    out_text += str(1.8) + "\n" +"0.0\n"

out_file = open("01_simple.expected", "w")
out_file.write(out_text[:-1])

pass

In [24]:
!cat 02_module_in_reaction.stride

use DesktopAudio version 1.0

import Generators

signal Value {
    default: 0
}

module Add10 {
    ports: [
		mainOutputPort OutputPort {
			block: Output
			domain: OutputPort.domain
		}
		mainInputPort InputPort {
			block: Input
			domain: OutputPort.domain
		}
    ]
    streams: [
        Input + 10 >> Output;
    ]
}

reaction Adder {
    streams: [
        Value >> Add10() >> Value;
    ]
}

AudioIn >> Greater() >> Adder();

Value >> AudioOut[0];


In [25]:
out_text = ''

value = 0.0
def add():
    global value
    value += 10
    

for val1, val2 in zip(input1, input2):
    if val1 > val2:
        add()
    out_text += str(value) + "\n" +"0.0\n"

out_file = open("02_module_in_reaction.expected", "w")
out_file.write(out_text[:-1])

pass

In [26]:
cat 03_module_input.stride

use DesktopAudio version 1.0

import Generators

signal Value {
    default: 0
}

reaction Adder {
    streams: [
        Value + 10 >> Value;
    ]
}

AudioIn >> Greater() >> Adder();

Value >> AudioOut[0];


In [27]:
out_text = ''

value = 0.0
def add():
    global value
    value += 10
    

for val1, val2 in zip(input1, input2):
    if val1 > val2:
        add()
    out_text += str(value) + "\n" +"0.0\n"

out_file = open("03_module_input.expected", "w")
out_file.write(out_text[:-1])

pass

In [12]:
cat 04_instance_portproperties.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
	]
	blocks: [
		signal Counter { domain: OutputPort.domain }
		reaction HasOuterPortProperty {
			ports: [
				mainOutputPort OutputPort {
					block: ReactionOutput
				}
			]
			streams: [
				Counter + 1 >> Counter;
				Counter / OutputPort.rate >> ReactionOutput;
			]
		}
			
	]
	streams: [
		on >> HasOuterPortProperty() >> Output;
	]
	
}
	
Test() >> AudioOut[0];


In [13]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    
    out_text += str(counter/44100.0) + "\n" +"0.0\n"

out_file = open("04_instance_portproperties.expected", "w")
out_file.write(out_text[:-1])

pass

In [14]:
cat 05_outer_instance_portproperties.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
	]
	blocks: [
		signal Counter {}
		reaction HasOuterPortProperty {
			ports: [
				mainOutputPort OutputPort {
					block: ReactionOutput
				}
			]
			streams: [
				Counter + 1 >> Counter;
				Counter / OutputPort.rate >> ReactionOutput;
			]
		}
			
	]
	streams: [
		on >> HasOuterPortProperty() >> Output;
	]
	
}
	
Test() >> AudioOut[0];


In [15]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    out_text += str(counter/44100.0) + "\n" +"0.0\n"

out_file = open("05_outer_instance_portproperties.expected", "w")
out_file.write(out_text[:-1])

pass

In [16]:
cat 06_used_only_in_reaction.stride

use DesktopAudio version 1.0

module InReaction {
    ports: [
        mainOutputPort OutputPort {
            block: Output
        }
        mainInputPort InputPort {
            block: Input
        }
    ]
	blocks: [
        reaction PassThruReaction {
			ports: [
				mainOutputPort OutputPort {
					block: Output
				}
			]
            streams:    [
                Input >> Output;
            ]
        }
    ]
    streams: [
        on >> PassThruReaction() >> Output;
    ]
}


AudioIn[0] >> InReaction () >> AudioOut;

In [17]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    out_text += str(val1) + "\n" + str(val1) + "\n"

out_file = open("06_used_only_in_reaction.expected", "w")
out_file.write(out_text[:-1])

pass

In [5]:
cat 07_reaction_in_module.stride

use DesktopAudio version 1.0

import Generators

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
		mainInputPort InputPort {
			block: Input
		}
	]
	blocks: [
		reaction InternalReaction {
			ports: [
			]
			streams: [
				Value + 1 >> Value;
			]
		}
		signal Value { default: 0 domain: OutputPort.domain}
		switch ReactionSwitch {
			domain: OutputPort.domain
		}
	]
	streams: [
		[Input, 0] >> Greater() >> ReactionSwitch >> InternalReaction();
		Value >> Output;
	]
}

AudioIn[0] >> Test() >> AudioOut[0];


In [21]:
out_text = ''

value = 0

for val1, val2 in zip(input1, input2):
    if val1 > 0:
        value += 1
    out_text += str(value) + "\n0.0\n"

out_file = open("07_reaction_in_module.expected", "w")
out_file.write(out_text[:-1])

pass

In [20]:
cat 08_reaction_recursive_reference.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainInputPort InputPort {
			block: Input	
		}	
		mainOutputPort OutputPort {
			block: Output	
		}
	]
	blocks: [
		signal Dummy {}
		signal Dummy2 { domain: OutputPort.domain}
		reaction Process {
			streams: [
				Dummy >> Dummy2;
				on >> ProcessInner();
			]
		}
		reaction ProcessInner {
			streams: [
				Input >> Output;
			]
		}
	]
	streams: [
		on >> Process();
	]
}

AudioIn[0] >> Test() >> AudioOut[0];

In [8]:
!cat 09_reaction_in_module2.stride

use DesktopAudio version 1.0

import Generators

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
		mainInputPort InputPort {
			block: Input
		}
	]
	blocks: [
		reaction InternalReaction {
			ports: [
				mainOutputPort OutputPort {
					block: ReactOutput
				}
			]
			streams: [
				Value + 0.001 >> ReactOutput;
			]
		}
		signal Value { default: 0}
		switch ReactionSwitch {
			domain: OutputPort.domain
		}
	]
	streams: [
		[Input, 0.2] >> Greater() >> ReactionSwitch >> InternalReaction() >> Output;
	]
}

AudioIn[0] >> Test() >> Test() >> AudioOut[0];


In [10]:
out_text = ''

value = 0

value2 = 0

for val1, val2 in zip(input1, input2):
    if val1 > 0.2:
        value += 0.001
    if value > 0.2:
        value2 += 0.001
    out_text += str(value2) + "\n0.0\n"

out_file = open("09_reaction_in_module2.expected", "w")
out_file.write(out_text[:-1])

pass

In [11]:
!cat 10_reaction_in_module_input_trigger.stride

use DesktopAudio version 1.0

module AD {
    ports:      [
        mainOutputPort OutputPort {
            name:       'output'
            block:      Output
            meta:       "The module's primary output port."
        }
        mainInputPort InputPort {
            name:       'output'
            block:      TriggerEnv
            meta:       "Triggers the envelope"
        }
    ]
    blocks:     [
        signal Output {
            default:    0.0
        }

        trigger TriggerEnv {domain: OutputPort.domain}
        signal Stage {default: -1 domain: OutputPort.domain}
    ]
    streams: [
	Stage + 0.2 >> Stage >> Output;
        TriggerEnv >> reaction { 0 >> Stage; };
    ]
}

[AudioIn[0], 0] >> Greater() >>  AD() >> AudioOut[0];

In [12]:
out_text = ''

stage = -1

for val1, val2 in zip(input1, input2):
    stage += 0.2
    out = stage
    if val1 > 0:
        stage = 0
    out_text += str(out) + "\n0.0\n"

out_file = open("10_reaction_in_module_input_trigger.expected", "w")
out_file.write(out_text[:-1])

pass

In [13]:
!cat 11_port_property.stride

use DesktopAudio version 1.0

module PortProperty {
    ports:      [
        mainOutputPort OutputPort {
            block:      Output
        }
        mainInputPort InputPort {
            block:      Input
        }
    ]
    blocks:     [
    ]
    streams: [
	[Input, 0] >> Greater() >> reaction { Input * InputPort.rate >> Output; };
    ]
}

AudioIn[0]>> PortProperty() >>  AudioOut[0];

In [14]:
out_text = ''

out = 0

for val1, val2 in zip(input1, input2):
    if val1 > 0:
        out = val1 * 44100
    out_text += str(out) + "\n0.0\n"

out_file = open("11_port_property.expected", "w")
out_file.write(out_text[:-1])

pass

In [16]:
!cat 13_anonymous_reaction.stride

use DesktopAudio version 1.0

switch AlwaysOn {
	default: on
	domain: AudioDomain
}

signal Value {
  domain: AudioDomain
}

AlwaysOn >> reaction {1.8 >> Value;};
Value >> AudioOut[0];
2.0 >> Value;
Value >> AudioOut[1];
	


In [18]:
out_text = ''

for val1, val2 in zip(input1, input2):
    out_text += "1.8\n2.0\n"

out_file = open("13_anonymous_reaction.expected", "w")
out_file.write(out_text[:-1])

pass